In [1]:
## Imports
import pandas as pd
import numpy as np
import time
import re

import spacy
from collections import Counter

np.random.seed(3000)

print("Imports completed")


Imports completed


In [2]:
LANGUAGE = 'fre'
SOURCE_DATASET = 'WiViCov2'

OUTPUT_DATASET_SIZE = 5000
NUM_OF_CANDIDATE_SETS = 10

SHORT_OUTPUT_DATASET_SIZE = 200

filter_out_problematic_instances = True
clean_up = True
measure_syntactic_diversity = True
measure_lexical_diversity = True


execution_start_time = time.time()
execution_start_time_string = time.strftime("%m_%d_%Hh")

output_file_name = "data_subset_" + LANGUAGE + "_" + str(OUTPUT_DATASET_SIZE) + "_from_" + SOURCE_DATASET + "_" + execution_start_time_string
print("Output file name is going to be: ", output_file_name)

Output file name is going to be:  data_subset_fre_5000_from_WiViCov2_08_31_10h


In [3]:
## Get the aligned sentences

source_file = '../../../datasets_original/wivico_dataset_v2/wivico_dataset_v2.tsv'
original_df = pd.read_csv(source_file, sep='\t', header=None, names=['wiki_sent', 'viki_sent', 'wiki', 'viki', 'pair'], skiprows=[0])
original_df


,wiki_sent,viki_sent,wiki,viki,pair
0,Catharanthus roseus La Pervenche de Madagascar...,"La pervenche de Madagascar (nom commun), ou Ca...",0,0,0
1,"Claude de France (Romorantin, 13 octobre 1499 ...",Claude de France est née le 13 octobre 1499 à ...,0,1,0
2,"Hippocrate de Kos Hippocrate de Kos, ou simple...",Hippocrate de Cos (surnommé Hippocrate le Gran...,0,0,0
3,"‌L'ASM Clermont Auvergne, anciennement Associa...","L'ASM Clermont Auvergne, anciennement AS Montf...",0,0,0
4,"‌L'ASM Clermont Auvergne, anciennement Associa...","L'ASM Clermont Auvergne, anciennement AS Montf...",0,0,0
...,...,...,...,...,...
46520,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...","Өвөр Монгол, Öwör Mongol ; en chinois : 内蒙古) e...",0,0,0
46521,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...",La Région autonome de Mongolie-Intérieure ou s...,0,0,0
46522,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...",La Région autonome de Mongolie-Intérieure ou s...,0,0,0
46523,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...",La Région autonome de Mongolie-Intérieure ou s...,0,0,0


In [4]:
relevant_df = original_df[(original_df['wiki'] == 0) &
                 (original_df['viki'] == 1) &
                 (original_df['pair'] == 0)]

relevant_df


,wiki_sent,viki_sent,wiki,viki,pair
1,"Claude de France (Romorantin, 13 octobre 1499 ...",Claude de France est née le 13 octobre 1499 à ...,0,1,0
10,"Le musée départemental Arles antique, dit « le...",Le musée de l'Arles antique est un musée d'arc...,0,1,0
11,"Le nahuatl (/ˈnaːwatɬ/ Écouter), dont le nom d...",Le nahuatl est la langue amérindienne la plus ...,0,1,0
13,Les Histoires comme ça (Just So Stories for Li...,Histoires comme ça est une série d'histoires p...,0,1,0
14,Les Histoires comme ça (Just So Stories for Li...,Histoires comme ça est une série d'histoires p...,0,1,0
...,...,...,...,...,...
46505,Zwolle [zvɔl(ə)] ou [zwɔl(ə)] ( prononcé en né...,"Zwolle est une ville des Pays-Bas, chef-lieu d...",0,1,0
46506,Zwolle [zvɔl(ə)] ou [zwɔl(ə)] ( prononcé en né...,"Zwolle est une ville des Pays-Bas, chef-lieu d...",0,1,0
46515,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...","Өвөр Монгол, Öwör Mongol ; en chinois : 内蒙古) e...",0,1,0
46517,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...","Өвөр Монгол, Öwör Mongol ; en chinois : 内蒙古) e...",0,1,0


In [5]:
if filter_out_problematic_instances:

    def has_multiple_sentences(text):
        # Match sentence-ending punctuation followed by a space and a capital letter (basic sentence structure)
        return bool(re.search(r'[.!?]\s+[A-ZÀ-Ý]', text))

    def contains_artifacts(text):   ## do not call for French
        # Define a regex pattern to match common artifact patterns
        # This pattern checks for unusual characters or sequences that are likely artifacts
        artifact_pattern = re.compile(r'[^\x20-\x7E]')  # Matches non-printable or unusual ASCII characters
        return bool(artifact_pattern.search(text))
            
    def contains_many_digits(text, threshold_base=5, threshold_increment=0.1):
        threshold = threshold_base + threshold_increment * len(text)
        digits = len(re.findall(r'\d', text))
        return digits > threshold
    
    def contains_many_special_chars(text, threshold_base=3, threshold_increment=0.1):
        threshold = threshold_base + threshold_increment * len(text)
        # Regex to match any character that is not a letter, digit, or common punctuation, including French accented characters
        special_chars = len(re.findall(r'[^a-zA-Z0-9\sÀ-ž]', text))
        return special_chars > threshold
    
    
    
    
    filtered_indices = []
    counter = 0
    sents_w_artifacts = 0
    sents_w_many_spec_chars = 0
    sents_w_many_digits = 0
    sents_that_are_multiple_sents = 0
    
    
    # Iterate over the DataFrame and find rows with artifacts
    for index, row in relevant_df.iterrows():
        sentence = row['wiki_sent']
        counter += 1

        if has_multiple_sentences(sentence):
            sents_that_are_multiple_sents += 1
            filtered_indices.append(index)
            if counter % 30 == 0:
                print("|| Found multiple sentences in instance " + str(index) + " ||")
                print(sentence)

        elif contains_many_digits(sentence):
            sents_w_many_digits += 1
            filtered_indices.append(index)
            if counter % 20 == 0:
                print("|| Too many digits in instance " + str(index) + " ||")
                print(sentence)

            
        else:
            if contains_many_special_chars(sentence):
                sents_w_many_spec_chars += 1
                filtered_indices.append(index)
                if counter % 10 == 0:
                    print("|| Too many special characters in instance " + str(index) + " ||")
                    print(sentence)         
            
            
    
    # Create a new DataFrame excluding the rows with artifacts
    clean_df = relevant_df.drop(index=filtered_indices)
    df = clean_df
    
    print("Filtering complete")
    #print("Filtered out ", sents_w_artifacts, " sentences containing artifacts.")
    print("Filtered out ", sents_that_are_multiple_sents, " instances with multiple sentences.")
    print("Then filtered out ", sents_w_many_digits, " sentences containing many digits.")
    print("Then filtered out ", sents_w_many_spec_chars, " sentences containing too many special characters.")



|| Found multiple sentences in instance 306 ||
À l'Ouest, rien de nouveau (titre original : en allemand : Im Westen nichts Neues) est un roman de Erich Maria Remarque paru en . Le roman décrit la Première Guerre mondiale vue par un jeune soldat volontaire allemand sur le front ouest.
|| Found multiple sentences in instance 439 ||
Abrictosaurus consors Abrictosaurus est un genre éteint de petits dinosaures ornithischiens primitifs appartenant à la famille des hétérodontosauridés. Il a vécu dans le Sud de l'Afrique , au Lesotho, où il a été découvert dans la formation géologique d'Elliot datée du Jurassique inférieur (Hettangien à Sinémurien), soit il y a environ entre 201,3 à 190,8 millions d'années.
|| Found multiple sentences in instance 528 ||
Addax nasomaculatus L'addax (Addax nasomaculatus) ou antilope à nez tacheté, est une espèce d'antilopes d'un genre monospécifique Addax et appartenant à la famille des Bovidés.  C'est une espèce endémique de l'Afrique, quasiment éteinte à l'éta

In [6]:
print(len(relevant_df))
print(len(clean_df))
print(len(relevant_df)-len(clean_df))
print(len(clean_df)/len(relevant_df))

20425
11610
8815
0.5684210526315789


In [7]:
if clean_up:

    def replace_LRB_RRB(text):
        # Replace "-LRB-" with "(" and "-RRB-" with ")"
        text = text.replace('-LRB- ', '(')
        text = text.replace(' -RRB-', ')')
        return text

    def replace_wrong_quotation_marks(text):
        # Replace patterns like `` word(s) '' with "word(s)"
        return re.sub(r'``\s*(.*?)\s*\'\'', r'"\1"', text)
    
    def remove_spaces_before_punctuation_FRENCH(text):
        # Regex pattern to match a space between a word-like string and a period or comma.
        return re.sub(r'(\w)\s+([.,\'])', r'\1\2', text)


    df['wiki_sent'] = df['wiki_sent'].apply(replace_LRB_RRB)
    #df['wiki_sent'] = df['wiki_sent'].apply(replace_wrong_quotation_marks)
    df['wiki_sent'] = df['wiki_sent'].apply(remove_spaces_before_punctuation_FRENCH)
    
    print("Clean up performed.")

else:
    print("No clean up")


Clean up performed.


In [8]:
counter = 0
for sentence in df['wiki_sent']:
    counter += 1
    if counter % 100 == 0:
        print(sentence)

Abu Bakr Mohammad Ibn Zakariya al-Razi, connu aussi comme Razi (persan : رازی ) ou Al-Razi, ou Ar-Razi, ou Ibn Zakaria (Zakariya) ou (en latin) comme Rhazes et Rasis, ou Rhasès (865-925) est un savant pluridisciplinaire perse qui a fait d'importantes contributions à la médecine, à l'alchimie et à la philosophie.
Albator (ハーロック, Hārokku?, Harlock en version originale) est un personnage de fiction créé par Leiji Matsumoto en 1969 dans le manga Dai-kaizoku Harlock.
Amiens (/a.mjɛ̃/[a]) est une commune française, préfecture du département de la Somme en région Hauts-de-France.
Anton Drexler (13 juin 1884 à Munich – 24 février 1942 à Munich) est un homme politique allemand d'extrême droite, fondateur du NSDAP et dirigeant du parti de 1919 à 1921, mais rapidement évincé ensuite par Adolf Hitler.
Aston Martin est un constructeur automobile britannique de voitures de luxe et de course, crée en 1913 par Lionel Martin et Robert Bamford.
Avec ses 316 km2 de superficie, c'est le plus petit État de

In [9]:
def get_dependency_tree(doc):
    # Get the dependency structure in terms of (head, relation, dependent)
    return [(token.head.dep_, token.dep_, token.dep_) for token in doc if token.head != token]

def calculate_tree_diversity(parse_trees):
    # Flatten the list of trees and count unique structures
    flattened_trees = [tuple(tree) for trees in parse_trees for tree in trees]
    tree_counter = Counter(flattened_trees)
    
    # Diversity score: higher score means more unique structures
    diversity_score = len(tree_counter) / sum(tree_counter.values())
    
    return diversity_score

In [10]:
# Load the English model
nlp = spacy.load("fr_core_news_lg")

candidate_sets = [df["wiki_sent"].sample(n=OUTPUT_DATASET_SIZE, replace=False) for _ in range(NUM_OF_CANDIDATE_SETS)]

sets_w_scores = []

start_time_all = time.time()

for idx, candidate_set in enumerate(candidate_sets):

    start_time = time.time()

    print("Assessing set ", idx+1, "/", NUM_OF_CANDIDATE_SETS, " of len ", OUTPUT_DATASET_SIZE)

    # List of sentences
    sentences = candidate_set

    print(type(sentences))

    # Parse the sentences
    docs = [nlp(sentence) for sentence in sentences]

    print(len(docs))

    tree_diversity_score = None
    std_clause_density_avg = None
    std_token_prob_avg = None



    if measure_syntactic_diversity:
        # Extract parse trees for all sentences
        parse_trees = [get_dependency_tree(doc) for doc in docs]
    
        elapsed_time_trees = time.time() - start_time
        print("Done with the parse_trees after ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time_trees)) + f".{int((elapsed_time_trees % 1) * 1000):03d}")
    
        # Calculate syntactic diversity score
        
        tree_diversity_score = calculate_tree_diversity(parse_trees)

    
        clause_density_avg_list = []
        for doc in docs:
            num_tokens = len([token for token in doc if token.is_alpha])
            num_clauses = sum(1 for token in doc if token.dep_ in ('ROOT', 'csubj', 'csubjpass', 'advcl', 'relcl', 'xcomp', 'ccomp'))
            clause_density = num_clauses / num_tokens if num_tokens else 0
            clause_density_avg_list.append(clause_density)
        
        std_clause_density_avg = np.std(clause_density_avg_list)

    if measure_lexical_diversity:
        token_prob_avg_list = []
        for doc in docs:
            # Calculate lexical diversity
            token_freqs = [token.prob for token in doc if token.is_alpha]  # Use token.prob as a proxy for frequency score
            token_prob_avg = np.mean(token_freqs) if token_freqs else 0
            token_prob_avg_list.append(token_prob_avg)
        
        std_token_prob_avg = np.std(token_prob_avg_list)
        
    
    sets_w_scores.append((idx, tree_diversity_score, std_clause_density_avg, std_token_prob_avg))
    
    elapsed_time_set = time.time() - start_time
    print("Finished set ",idx, "after ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time_set)) + f".{int((elapsed_time_set % 1) * 1000):03d}")

elapsed_time_all = time.time() - start_time_all
print("Finished all after ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time_all)) + f".{int((elapsed_time_all % 1) * 1000):03d}")
print(sets_w_scores)

Assessing set  1 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:02:47.124
Finished set  0 after  00:02:48.305
Assessing set  2 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:02:06.715
Finished set  1 after  00:02:07.692
Assessing set  3 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:01:31.717
Finished set  2 after  00:01:32.616
Assessing set  4 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:01:03.161
Finished set  3 after  00:01:03.613
Assessing set  5 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:00:46.088
Finished set  4 after  00:00:46.576
Assessing set  6 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:00:47.273
Finished set  5 after  00:00:47.728
Assessing set  7 / 10  of len  5000
<class 'pa

In [11]:
# Weights
weights = [0.3, 0.3, 0.4]

# Function to calculate the weighted sum for a tuple
def weighted_sum(tup):
    return weights[0] * tup[1] + weights[1] * tup[2] + weights[2] * tup[3]

# Find the tuple with the maximum weighted sum
best_tuple = max(sets_w_scores, key=weighted_sum)

print(sets_w_scores)
print(best_tuple)

[(0, 0.003934918591784658, 0.052935057040355143, 0.0), (1, 0.004028051304653459, 0.05249580237022374, 0.0), (2, 0.0039944033218992595, 0.054817609712976764, 0.0), (3, 0.003977074001782632, 0.05195814338257731, 0.0), (4, 0.0039982355735517, 0.05166714473236815, 0.0), (5, 0.003940831074977416, 0.053237781864617494, 0.0), (6, 0.0040707934664328685, 0.05300529398371937, 0.0), (7, 0.004044277771178723, 0.05317969799053528, 0.0), (8, 0.0039990271108018985, 0.05293894486990884, 0.0), (9, 0.003986957917321305, 0.05383677756169871, 0.0)]
(2, 0.0039944033218992595, 0.054817609712976764, 0.0)


In [12]:
best_set_of_sentences = candidate_sets[best_tuple[0]]
print(best_set_of_sentences)

37241    Panurus biarmicus La Panure à moustaches (Panu...
35958    Monsieur Wens, surnom de Wenceslas Vorobeïtchi...
13955    Ils permettent aux enfants de s'amuser en fais...
39745    Saint-Ignace-de-Loyola est une municipalité du...
25619    Le hanafisme ou hanéfisme (en arabe : حنفي, Ḥa...
                               ...                        
6778     Dark Vador Anakin Skywalker, ou Dark Vador (Da...
44496    Une calotte glaciaire est un type de glacier f...
44368    Une anagramme est un mot ou une expression obt...
38187    Pomme étoile, Caïmite La caïmite, aussi appelé...
43181    Un ennéagone, ou nonagone,,, est un polygone à...
Name: wiki_sent, Length: 5000, dtype: object


In [13]:
with open(output_file_name, "w") as file:
    counter = 0
    for index, sentence in best_set_of_sentences.items():
        counter += 1
        file.write(f"{index}|{sentence}\n\n")
        if counter % 50 == 0:
            print(f"{index}|{sentence}\n\n")
        

43605|Un miracle est un fait extraordinaire, dépourvu d'explication scientifique, qui est alors vu comme surnaturel et attribué à une puissance divine.


13944|Ils étaient initialement cinq, avec Smaïn et Seymour Brussel qui quittent le groupe un peu plus d'un an après sa création.


45525|Valentin Brunel, dit Kungs (prononcé : [kuŋz]), est un disc jockey, auteur-compositeur et musicien français, né le 17 décembre 1996 à Toulon.


5327|Christian Marie Dominique Liberté Boltanski, né le 6 septembre 1944 à Paris 7e et mort à  Paris 14e le 14 juillet 2021, est un artiste plasticien français reconnu comme l'un des principaux artistes français contemporains.


38296|Pouic-Pouic est un film français réalisé par Jean Girault, sorti en 1963, d'après la pièce de théâtre Sans cérémonie de Jacques Vilfrid et Jean Girault, créée en 1952.


18974|La Jeanneke-Pis, signifiant « la petite fille [qui] pisse » en bruxellois, est une statue avec fontaine située à Bruxelles en Belgique.


10087|Étampes (p

In [14]:
short_output_file_name = "data_subset_" + LANGUAGE + "_" + str(SHORT_OUTPUT_DATASET_SIZE) + "_from_" + SOURCE_DATASET + "_" + execution_start_time_string

short_sample = best_set_of_sentences.sample(n=SHORT_OUTPUT_DATASET_SIZE, replace=False)

with open(short_output_file_name, "w") as file:
    counter = 0
    for index, sentence in short_sample.items():
        counter += 1
        file.write(f"{index}|{sentence}\n\n")
        if counter % 100 == 0:
            print(f"{index}|{sentence}\n\n")


20560|La profiterole, parfois mal orthographiée profiterolle est une pâtisserie sucrée, composée à l'origine d'un petit chou rempli de crème pâtissière, fleurette ou chantilly et recouvert très souvent d'une sauce au chocolat dans la préparation « profiteroles au chocolat ».


27975|Le Pilat est un massif montagneux situé à l'est du Massif central, qui culmine au crêt de la Perdrix à 1 431 mètres d'altitude, constituant une région naturelle française.




In [15]:
execution_elapsed_time = time.time() - execution_start_time
print("Code execution completed after ", time.strftime("%H:%M:%S", time.gmtime(execution_elapsed_time)) + f".{int((execution_elapsed_time % 1) * 1000):03d}")

Code execution completed after  07:38:42.159
